<a href="https://colab.research.google.com/github/KonstantinBurkin/Math_for_DS_exam/blob/main/Optimisation_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<center><h1>Optimisation algorithms</h1></center>
<br>

<center><h3>Konstantin Burkin</h3></center>

<center> <i>27 March 2022 </i> </center>

<br>  
<h2>Table of contents</h2> 

---
- <a href=#Introduction>Introduction</a> <br>
  - <a href=#Brief-outline>Brief outline</a> <br>
- <a href=#Import-of-data-and-Python-libraries>Setup of notebook environment</a> <br>
  - <a href=#Conditions-of-the-problem>Conditions of the problem</a> <br>
- <a href=#Gradient-descent>1. Gradient descent</a> <br>
- <a href=#Heavy-ball-method>2. Heavy ball method</a> <br>
- <a href=#Accelerated-gradient-descent>3. Accelerated gradient descent</a> <br>
- <a href=#Newton-method>4. Newton method</a> <br>
- <a href=#Results>Results</a> <br>

<br> 
<h2 id="Introduction">Introduction</h2>  

--- 

<div align="justify">This work was created and submitted as an exam for MSU course "Math for Data Science" in spring semester 2022. The project was written in <a href="https://https://colab.research.google.com">Google Colab</a>, using Python <i>version 3.7.13</i>. This work is available in <a href="https://github.com/KonstantinBurkin/Math_for_DS_exam">my Github repository</a>, where it is possible to download Colab Notebook and see the code.</div> <br>

<div align="justify">The goal is to solve 4 problems and submit the result to the <a href="https://contest.yandex.ru/">Yandex.contest</a> platform. It was required to program optimisation algorithms (gradient descent, heavy ball method, accelarated gradient descent, Newton method) and calculate the number of iterations needed to meet the condition of the problem.

<br>  

<h3 id="Brief-outline">Brief outline</h3> 

<ul>
  <li>Data import, setting up notebook environment: import of libraries and setting up double-precision floating-point format (64 bit).</li>
  <li>Description of function and initial conditions</li>
  <li>Brief description of gradient descent, the problem conditions and the solution to the problem</li>
  <li>Brief description of heavy ball method, the problem conditions and the solution to the problem</li>
  <li>Brief description of accelerated gradient descent, the problem conditions and the solution to the problem</li>
  <li>Brief description of Newton method, the problem conditions and the solution to the problem</li>
</ul> 

</div> 

<br> 
<h2 id="Import-of-data-and-Python-libraries">Setup of notebook environment</h2>  

--- 

In [1]:
# Import libraries
import numpy as np
import scipy.optimize as scopt
import warnings
import jax.numpy as jnp
import jax
from jax.config import config
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Settings configuration
warnings.filterwarnings('ignore')                      
config.update("jax_enable_x64", True)               # provides float64 accuracy

<div align="justify">The following data analysis includes several Python libraries for data analysis, builduing ML models, ploting data, etc: </div>  

- Numpy
- Scipy
- Jax
- Plotly

<div align="justify">The environment of the notebook was set to float64 precision.</div>
<br>  

<h3 id="Conditions-of-the-problem">Conditions of the problem</h3> 

<div align="justify">
Objective: Solve 4 problems using the example of the \(f(x)\) function minimization.<br><br>
$$f(x)\ =\ ∣∣Ax−b∣∣^3_{2} $$<br>  
m = 100, n = 10, np.random.seed(0);<br>
Generate matrix: A = np.random.randn(m, n).<br>
Generate matrix: b = np.random.randn(m).<br>
The initial approximation is the zero vector: \(x_0\) = jax.numpy.zeros(n) <br>
</div>
  
<!-- Function: $f(x)\ =\ ∣∣Ax−b∣∣^3_{2}$ -->

In [ ]:
# conditions of the problem

np.random.seed(0)

m = 100
n = 10
A = np.random.randn(m, n)     # generate matrix 
b = np.random.randn(m)        # generate vector

x0 = jax.numpy.zeros(n)       # initial approximation
eps = 1e-4                    # accuracy
alpha = 1e-4                  # constant step 
num_iter = 2000               # max number of iterations

f = lambda x: (jax.numpy.linalg.norm(A@x - b))**3
gradf = jax.grad(f)

<br> 
<h2 id="Gradient-descent">1. Gradient descent</h2>  

--- 

<div align="justify">Description of algorithm:<br>
This iterative optimization algorithm is used for finding a local minimum of a differentiable function by taking repeated steps in the opposite direction of the gradient.</div>
  
$$x_{n+1} = x_n - α * \nabla F(x_n)$$
  
<div align="justify">The task:<br>
How many iterations does the gradient descent method with a constant step of \(α = 10^{-4} \) take to achieve an accuracy in gradient norm less than \(10^{-4} \) ?</div>

In [3]:
# create gradient descent algorithm
def grad_descent(f, grad, x0, num_iter, eps, alpha):
    x = x0.copy()
    conv = [x]
    norm_gd = []
    for i in range(num_iter):
        h = -grad(x)
        norm_gd.append(np.linalg.norm(h))
        if jnp.linalg.norm(h) < eps:
            break
        x = x + alpha * h
        conv.append(x)
    return x, conv, i, norm_gd

# result of gradient descent algorithm
x_gd, conv_gd, i_gd, norm_gd = grad_descent(f, gradf, x0, num_iter, eps, alpha)
# print("Answer: ",i_gd+1, " iterations.\n")             # +1 b/c count from 0    

# plot graph
fig = go.Figure(go.Scatter( y=[x for x in norm_gd] ))

# modify layout of the graph 
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  yaxis=dict(type="log", tickformat = '10.0e'),
                  title={'text': "Convergence of gradient norm", 'y':0.955, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Iterations',
                  yaxis_title='log (Gradient norm)',
                  showlegend=False,
                  margin=dict(l=20, r=20, t=60, b=20),
                  )

fig.show()

Answer:  88  iterations.

<br> 
<h2 id="Heavy-ball-method">2. Heavy ball method</h2>  

--- 

<div align="justify">Description of algorithm:<br>
This algorithm, also known as Polyak’s momentum, combines the gradient descent with a history of the previous step to accelerate the convergence of the algorithm.</div>

$$x_{n+1} = x_n - α * \nabla F(x_n) + β(x_n-x_{n-1})$$
  
<div align="justify">The task:<br>
How many iterations does the heavy ball method with a constant step of \(α = 10^{-4} \) and coefficient \(β = 0.5 \) take to achieve an accuracy in the gradient norm less than \(10^{-4} \) ?</div>
<!-- Function: $f(x)\ =\ ∣∣Ax−b∣∣^3_{2}$ -->

In [4]:
# create heavy ball algorithm
def heavy_ball(f, grad, x0, num_iter, eps, alpha, beta):
    x = x0.copy()
    conv = [x.copy()]
    prev_x = None
    norm_hb = []
    for i in range(num_iter):
        h = -grad(x)
        norm_hb.append(np.linalg.norm(h))
        if jax.numpy.linalg.norm(h) < eps:
          break
        if prev_x is None:
            prev_x = x
            x = x + alpha * h
        else:
            x, prev_x = x + alpha * h + beta * (x - prev_x), x
        conv.append(x.copy())
    return x, conv, i, norm_hb

# result of heavy ball algorithm
x, conv_hb, i, norm_hb = heavy_ball(f, gradf, x0, num_iter, eps, alpha, beta=0.5)
# print("Answer: ",i+1, " iterations.\n")              # +1 b/c count from 0  

# plot graph
fig = go.Figure(go.Scatter( y=[x for x in norm_hb] ))

# modify layout of the graph 
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  yaxis=dict(type="log", tickformat = '10.0e'),
                  title={'text': "Convergence of gradient norm", 'y':0.955, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Iterations',
                  yaxis_title='log (Gradient norm)',
                  showlegend=False,
                  margin=dict(l=20, r=20, t=60, b=20),
                  )

fig.show()

Answer:  47  iterations.

<br> 
<h2 id="Accelerated-gradient-descent">3. Accelerated gradient descent</h2>  

--- 

<div align="justify">Description of algorithm:<br>
This algorithm, also known as Nesterov Momentum, is an extension to the gradient descent optimization algorithm. It makes the gradient descent
with reference to the approximate future position.</div>

$$y_0 = x_0 $$
$$x_{n+1} = y_{n} - α * \nabla F(y_n) $$
$$y_{n+1} = x_{n+1} + β(x_{n+1}-x_n)$$
  
<div align="justify">The task:<br>
How many iterations does the accelerated gradient method with a constant step of \(α = 3* 10^{-4} \) take to achieve an accuracy in gradient norm less than \(10^{-4}\) ?</div>

In [5]:
alpha = 3e-4              # new step

# create accelerated gradient descent algorithm
def accelerated_gd(f, grad, x0, num_iter, eps, alpha):
    x = x0.copy()
    y = x0.copy()
    conv = [x]
    norm_acc_gd = []
    prev_x = None
    for i in range(num_iter):
        h = -grad(y)
        norm_acc_gd.append(np.linalg.norm(h))
        if np.linalg.norm(h) < eps:
            break
        if prev_x is None:
            prev_x = x.copy()
            x = y + alpha * h
        else:
            x, prev_x = y + alpha * h, x
        
        y = x + (i+1.) / (i+4.) * (x - prev_x)
            
        conv.append(x)
    return x, conv, i, norm_acc_gd

# result of accelerated gradient descent algorithm
x_acc_gd, conv_acc_gd, i_acc_gd, norm_acc_gd = accelerated_gd(f, gradf, x0, num_iter, eps, alpha)
# print("Answer: ",i_acc_gd+1, " iterations.\n")             # +1 b/c count from 0    

# plot graph
fig = go.Figure(go.Scatter( y=[x for x in norm_acc_gd] ))

# modify layout of the graph 
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  yaxis=dict(type="log", tickformat = '10.0e'),
                  title={'text': "Convergence of gradient norm", 'y':0.955, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Iterations',
                  yaxis_title='log (Gradient norm)',
                  showlegend=False,
                  margin=dict(l=20, r=20, t=60, b=20),
                  )

fig.show()

Answer:  33  iterations.

<br> 
<h2 id="Newton-method">4. Newton method</h2>  

--- 

<div align="justify">Description of algorithm:<br>
 Newton's method is an iterative method for finding a local minimum of a twice-differentiable function. At each iteration, it amounts to the fitting of a parabola to the graph of \(f(x)\).</div>

$$x_{n+1} = x_n - \frac {F^{'}(x_n)}{ F^{''}(x_n)}$$

<div align="justify">The task:<br>
After how many iterations does Newton's method with a constant step of \(α = 10^{-1} \) begins to stagnate, that is, the gradient norm stops changing? In your answer, indicate the number of iteration k such that the value of the gradient norm at the next iteration k+1 remains the same.<br>
What rate of convergence of Newton's method is observed in the norm of the gradient?</div>

In [10]:
from jax.config import config                       
config.update("jax_enable_x64", False)               # set back to float32

# restate new conditions
np.random.seed(0)

m = 100
n = 10
A = np.random.randn(m, n)     # generate matrix 
b = np.random.randn(m)        # generate vector

x0 = jax.numpy.zeros(n)       # initial approximation
eps = 1e-4                    # accuracy
alpha = 1e-1                  # constant step 
num_iter = 500               # max number of iterations

f = lambda x: (jax.numpy.linalg.norm(A@x - b))**3
gradf = jax.grad(f)

# create Newton algorithm
def newton_minimize(f, gradf, hessf, x0, alpha, num_iter):
    x = x0.copy()
    conv = [x]
    norm = []
    prev_norm = None
    for i in range(num_iter):
        H = hessf(x)
        g = gradf(x)
        x = x - alpha * jax.numpy.linalg.solve(H, g)

        norm.append(np.linalg.norm(g))

        if norm[-1] == prev_norm:
            # -1 b/c started to stagnate on previous step
            # +1 b/c count from 0
            break

        prev_norm = norm[-1]
        conv.append(x)
    return x, conv, norm, i

# result of Newton algorithm
hess = jax.hessian(f)
x, conv, norm, i = newton_minimize(f, gradf, hess, x0, alpha, num_iter)
# print("Answer: ",i, "iterations.\nLinear rate of convergence is observed.\n")

# plot graph
fig = go.Figure(go.Scatter( y=[x for x in norm] ))

# modify layout of the graph 
fig.update_layout(
                  yaxis=dict(type="log", tickformat = '10.0e'),
                  font_family="'Nunito', sans-serif",
                  title={'text': "Convergence of gradient norm", 'y':0.955, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Iterations',
                  yaxis_title='log (Gradient norm)',
                  showlegend=False,
                  margin=dict(l=20, r=20, t=60, b=20),
                  )

fig.show()

Answer:  144 iterations.<br>
Linear rate of convergence is observed.

<br> 
<h2 id="Results">Results</h2>  

--- 

<div align="justify">In this project, 4 problems concerning function minimisation were solved. 4 optimisation algorithms were used: gradient descent, heavy ball method, accelerated gradient descent, and Newton Method.
</div>
  

<br><br><br>
<center><a href="https://konstantinburkin.github.io">Main page</a></center><br><br>